In [1]:
from src.data.loader import DLConfig, DataLoader

In [2]:
config = DLConfig(
    data_loc = 'test/data',
    batch_size = 16,
    #_sim_shard = (0, 8),
)
loader = DataLoader('ssv2', config)

In [3]:
from tqdm import tqdm

for i in tqdm(loader.train_loader()):
    pass

71it [00:02, 26.79it/s]


KeyboardInterrupt: 

In [4]:
i[1].shape

torch.Size([16, 32, 3, 224, 224])